In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
from absl_extra import logging_utils
from temporal_fusion_transformer.src.datasets import downcast_dataframe
from datetime import datetime
from absl import logging

logging_utils.setup_logging()

start_date = datetime(2016, 5, 1)
end_date = datetime(2016, 6, 1)

In [3]:
lazy_temporal = pl.scan_csv("../data/favorita/train.csv", try_parse_dates=True)
lazy_temporal = downcast_dataframe(lazy_temporal)

lazy_temporal = lazy_temporal.filter(pl.col("date") >= start_date)
lazy_temporal = lazy_temporal.filter(pl.col("date") <= end_date)
logging.debug("Adding trajectory identifier")
lazy_temporal = lazy_temporal.with_columns(
    [
        pl.format("{}_{}", "store_nbr", "item_nbr").alias("traj_id"),
    ]
)
# Remove all IDs with negative returns
logging.debug("Removing returns data")

lazy_temporal = lazy_temporal.filter(pl.col("unit_sales").min().over("traj_id") >= 0)
lazy_temporal = lazy_temporal.with_columns(open=pl.lit(1))
lazy_temporal.collect()

2023-08-27 21:45:09,053:[2607012406.py:6-><module>()]:DEBUG: Adding trajectory identifier
2023-08-27 21:45:09,053:[2607012406.py:13-><module>()]:DEBUG: Removing returns data


id,date,store_nbr,item_nbr,unit_sales,onpromotion,traj_id,open
i32,date,i32,i32,f32,str,str,i32
77852003,2016-05-01,1,103665,4.0,"""False""","""1_103665""",1
77852004,2016-05-01,1,105575,7.0,"""False""","""1_105575""",1
77852005,2016-05-01,1,105693,2.0,"""False""","""1_105693""",1
77852006,2016-05-01,1,105857,5.0,"""False""","""1_105857""",1
77852007,2016-05-01,1,106716,2.0,"""False""","""1_106716""",1
77852008,2016-05-01,1,108696,1.0,"""False""","""1_108696""",1
77852009,2016-05-01,1,108698,1.0,"""False""","""1_108698""",1
77852010,2016-05-01,1,108786,1.0,"""False""","""1_108786""",1
77852011,2016-05-01,1,108797,2.0,"""False""","""1_108797""",1


In [4]:
# LazyFrame does not support up-sampling
temporal = (
    lazy_temporal.sort("date", "traj_id")
    .collect()
    .upsample("date", every="1h", by="traj_id")
    .fill_null(strategy="forward")
)
temporal

date,id,store_nbr,item_nbr,unit_sales,onpromotion,traj_id,open
date,i32,i32,i32,f32,str,str,i32
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1
2016-05-01,77869579,8,1489850,52.761002,"""False""","""8_1489850""",1


In [5]:
temporal = temporal.with_columns([pl.col("unit_sales").log().alias("log_sales")]).drop(["unit_sales"])
temporal

date,id,store_nbr,item_nbr,onpromotion,traj_id,open,log_sales
date,i32,i32,i32,str,str,i32,f32
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773
2016-05-01,77869579,8,1489850,"""False""","""8_1489850""",1,3.965773


In [6]:
store_info = downcast_dataframe(pl.read_csv("../data/favorita/stores.csv"))
items = downcast_dataframe(pl.read_csv("../data/favorita/items.csv"))
transactions = downcast_dataframe(pl.read_csv("../data/favorita/transactions.csv", try_parse_dates=True))

In [18]:
oil = downcast_dataframe(
    pl.read_csv("../data/favorita/oil.csv", try_parse_dates=True)
    .sort("date")
    .with_columns(pl.col("dcoilwtico").fill_null(strategy="forward"))
    .filter(pl.col("dcoilwtico").is_not_null())
    .rename({"dcoilwtico": "oil_price"})
)
oil

date,oil_price
date,f32
2013-01-02,93.139999
2013-01-03,92.970001
2013-01-04,93.120003
2013-01-07,93.199997
2013-01-08,93.209999
2013-01-09,93.080002
2013-01-10,93.809998
2013-01-11,93.599998
2013-01-14,94.269997


In [7]:
holidays = downcast_dataframe(pl.read_csv("../data/favorita/holidays_events.csv", try_parse_dates=True))

In [8]:
national_holidays = (
    holidays.filter(pl.col("locale") == "National")
    .select(["description", "date"])
    .rename({"description": "national_hol"})
)

In [13]:
regional_holidays = (
    holidays.filter(pl.col("locale") == "Regional")
    .select(["description", "locale_name", "date"])
    .rename({"locale_name": "state", "description": "regional_hol"})
)
local_holidays = (
    holidays.filter(pl.col("locale") == "Local")
    .select(["description", "locale_name", "date"])
    .rename({"locale_name": "city", "description": "local_hol"})
)

In [ ]:
temporal: pl.DataFrame

df = (
    temporal.join(oil, on="date", how="left")
    .join(store_info, on="store_nbr")
    .join(items, on="item_nbr")
    .join(transactions, on=["store_nbr", "date"])
    .join(national_holidays, on="date", how="left")
    .join(regional_holidays, on=["date", "state"], how="left")
    .join(local_holidays, on=["date", "city"], how="left")
    .with_columns(
        [
            pl.col("oil_price").fill_null(strategy="forward"),
            pl.col("national_hol").fill_null(""),
            pl.col("regional_hol").fill_null(""),
            pl.col("local_hol").fill_null(""),
        ]
    )
)
df